In [16]:
def suma(a, b, c):
    return a+b+c

def resta(a,b, que=False):
    return a-b    

def gen_exp(funcion, **kwargs):
    return funcion(**kwargs)    

In [11]:
gen_exp(suma, a=1, b=2, c=3)

6

In [13]:
diccionario = {"a" : 1, "b" : 2}

In [17]:
resta(que=False, **diccionario)

-1

In [18]:
settings = [
    {"n_shots": 500, 
    "n_repeat": 10, 
    "no_particles": 150, 
    "SMC_fun": "default"},
    {
        "n_shots": 500,
        "n_repeat": 10,
        "no_particles": 150,
        "SMC_fun": "slower",
        "update_rate": 0.4,
    },
    {"n_shots": 500, 
    "n_repeat": 10, 
    "no_particles": 500, 
    "SMC_fun": "default"},
    {
        "n_shots": 500,
        "n_repeat": 10,
        "no_particles": 500,
        "SMC_fun": "slower",
        "update_rate": 0.4,
    },
]


In [19]:
import json

In [21]:
json.dumps(settings, "lista.json")

TypeError: dumps() takes 1 positional argument but 2 were given

In [29]:
with open("mydata.json", "w") as final:
    json.dump(settings, final)

In [31]:
with open("mydata.json") as final:
    a = json.load(final)


In [33]:
a[0]

{'n_shots': 500, 'n_repeat': 10, 'no_particles': 150, 'SMC_fun': 'default'}